In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_json("tweets_data.json")
data.head()

,token,pos,head,deprel,aspects
0,"[i, agree, about, arafat, ,, i, mean, ,, shit,...","[LS, VBP, IN, NN, ,, FW, NN, ,, NN, ,, PRP, RB...","[2, 0, 4, 2, 7, 7, 2, 7, 2, 2, 13, 13, 2, 13, ...","[nsubj, root, case, nmod, punct, nsubj, parata...","[{'term': ['jimmy', 'carter'], 'from': 15, 'to..."
1,"[musicmonday, britney, spears, -, lucky, do, y...","[NN, NN, NNS, :, JJ, VBP, PRP, VB, DT, NN, ,, ...","[3, 3, 0, 3, 3, 8, 8, 3, 10, 8, 8, 8, 8, 15, 3...","[amod, compound, root, punct, dep, aux, nsubj,...","[{'term': ['britney', 'spears'], 'from': 1, 't..."
2,"[wtf, ,, hilary, swank, is, coming, to, my, sc...","[NN, ,, JJ, NN, VBZ, VBG, TO, PRP$, NN, NN, ,,...","[6, 6, 4, 6, 6, 0, 9, 9, 6, 6, 6, 14, 14, 6, 1...","[advmod, punct, compound, nsubj, aux, root, ca...","[{'term': [',', 'hilary', 'swank'], 'from': 1,..."
3,"[my, 3-year-old, was, amazed, yesterday, to, f...","[PRP$, NN, VBD, VBN, NN, TO, VB, IN, '', JJ, '...","[2, 4, 4, 0, 4, 7, 4, 16, 10, 14, 14, 14, 14, ...","[nmod, nsubjpass, auxpass, root, nmod, mark, x...","[{'term': ['wii'], 'from': 21, 'to': 22, 'pola..."
4,"[God, damn, ,, That, Sony, remote, for, google...","[NNP, RB, ,, IN, NNP, JJ, IN, NN, VBZ, RB, JJ, .]","[0, 1, 1, 6, 6, 1, 8, 6, 10, 11, 1, 1]","[root, advmod, punct, mark, nsubj, amod, case,...","[{'term': ['google'], 'from': 7, 'to': 8, 'pol..."


In [4]:
X=pd.DataFrame(columns=['tweets'])
X['tweets']=data['token']
X['tweets']=X['tweets'].apply(lambda x:' '.join(x))
X

,tweets
0,"i agree about arafat , i mean , shit , they ev..."
1,musicmonday britney spears - lucky do you reme...
2,"wtf , hilary swank is coming to my school toda..."
3,my 3-year-old was amazed yesterday to find tha...
4,"God damn , That Sony remote for google is fuck..."
...,...
6046,"really , that seems like more of a britney spe..."
6047,the biggest loser wii review
6048,"ok love game is on , , lady gaga , my dear , y..."
6049,nic cage sues business manager for ' financial...


In [5]:
y=pd.DataFrame(columns=['polarity'])
for i in data['aspects']:
    for j in i:
        y=y._append({'polarity':j['polarity']},ignore_index=True)

In [6]:
y['polarity']

0       negative
1       positive
2        neutral
3        neutral
4       negative
          ...   
6046     neutral
6047    negative
6048    positive
6049     neutral
6050     neutral
Name: polarity, Length: 6051, dtype: object

In [7]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(drop='first')
y_new=ohe.fit_transform(y[['polarity']]).toarray()
y_new

array([[0., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [8]:
print(X.shape,y.shape)


(6051, 1) (6051, 1)


In [24]:
from keras_preprocessing.text import Tokenizer
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding
from keras.utils import pad_sequences 

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X=X['tweets']
y=y_new

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [13]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(4538,) (4538, 2)
(1513,) (1513, 2)


# Tokenizing the text

In [14]:
tokenizer=Tokenizer(num_words=999,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")

In [15]:
tokenizer.fit_on_texts(X)

In [16]:
X_train_tok=tokenizer.texts_to_sequences(X_train)
X_test_tok=tokenizer.texts_to_sequences(X_test)

In [17]:
X_test_tok[0]

[302, 264, 82, 776, 24, 185, 280, 469, 3, 1, 111, 2, 527, 5, 90, 564]

In [18]:
X_train.head()

332       katy perry wants to crash the royal wedding , .
833     when you compare lindsay lohan pre and post co...
2582    Wave An Amazing list of all the google product...
5709    industry games - guitar hero brand respected l...
2611    the wii sucks , its fun for a couple of weeks ...
Name: tweets, dtype: object

In [19]:
X_train_tok[0]

[274, 278, 391, 3, 2, 852]

In [20]:
X_train_pad=pad_sequences(X_train_tok)
X_test_pad=pad_sequences(X_test_tok)

In [21]:
X_train_pad[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0, 274, 278, 391,   3,   2, 852])

In [22]:
X_test_pad[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 302, 264,  82, 776,  24, 185, 280, 469,   3,   1, 111,
         2, 527,   5,  90, 564])

# Model Training

In [34]:
model=Sequential()
model.add(Embedding(2000,2))
model.add(SimpleRNN(64,return_sequences=False))
model.add(Dense(2,activation="softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 2)           4000      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 64)                4288      
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 8,418
Trainable params: 8,418
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [36]:
model.fit(X_train_pad,y_train,epochs=10,validation_data=[X_test_pad,y_test])

Epoch 1/10
142/142 [==============================] - 2s 10ms/step - loss: 0.4797 - accuracy: 0.7488 - val_loss: 0.4824 - val_accuracy: 0.7568
Epoch 2/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4790 - accuracy: 0.7490 - val_loss: 0.4782 - val_accuracy: 0.7568
Epoch 3/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4795 - accuracy: 0.7490 - val_loss: 0.4770 - val_accuracy: 0.7568
Epoch 4/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4797 - accuracy: 0.7490 - val_loss: 0.4758 - val_accuracy: 0.7568
Epoch 5/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4799 - accuracy: 0.7490 - val_loss: 0.4826 - val_accuracy: 0.7568
Epoch 6/10
142/142 [==============================] - 1s 9ms/step - loss: 0.4809 - accuracy: 0.7490 - val_loss: 0.4749 - val_accuracy: 0.7568
Epoch 7/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4790 - accuracy: 0.7490 - val_loss: 0.4757 - val_accuracy: 0.7568
Epoch